# Masterthesis

In [ ]:
import networkx as nx
import markovmixing as mkm
import numpy as np
import random

execfile('graph_util.py')

## Generate and save some graphs 

In [ ]:
random.seed(0)

# G = erdos_renyi_giant_component(5000000,1.05/5000000)
# print nx.number_of_nodes(G)

# G_2core = nx.k_core(G,k=2)
# print nx.number_of_nodes(G_2core)

# nx.write_sparse6(G_2core, 'ergc_2core_1.s6')

random.seed(1)

# G = erdos_renyi_giant_component(5000000,1.1/5000000)
# print nx.number_of_nodes(G)

# G_2core = nx.k_core(G,k=2)
# print nx.number_of_nodes(G_2core)

# nx.write_sparse6(G_2core, 'ergc_2core_2.s6')

## Determine mixing times on different types of graphs

### Lazy random walk on the 1000-path

In [ ]:
G = nx.path_graph(1000)

mc = mkm.nx_graph_lazy_srw(G)

for i in [0,250,500]:
    d = np.zeros(1000)
    d[i] = 1.
    mc.add_distributions(d)
    
mc.plot_tv_mixing(y_tol=0.01, threshold=0.05, text=False)

### Lazy biased random walk on the 1000-path

In [ ]:
mc = mkm.MarkovChain(mkm.line_lazy_transition_matrix(1000, p=0.51))

for i in [0,500,999]:
    d = np.zeros(1000)
    d[i] = 1.
    mc.add_distributions(d)
    
mc.plot_tv_mixing(y_tol=0.01, threshold=1e-5, text=False)

### Lazy random walk on the 50-cycle

In [ ]:
G = nx.cycle_graph(50)

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### Lazy random walk on the 50-cycle with appended binary trees of height 8

In [ ]:
G = append_graph_to_all_nodes(nx.cycle_graph(50), nx.balanced_tree(2,8), 0)

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### Lazy random walk on the 15-dimensional hypercube

In [ ]:
mc = mkm.MarkovChain(mkm.hypercube_transition_matrix(15))
mc.add_random_delta_distributions(1)
mc.set_stationary(mkm.uniform_distribution(mc.get_n()))

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### SRW on a random 6-regular graph with n=50.000

In [ ]:
G = nx.read_sparse6('6_regular.s6')

mc = mkm.nx_graph_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

In [ ]:
################ Mixing on the 2-core of the giant component of an Erdős–Rényi random graph ################

G = nx.read_sparse6('ergc_2core_1.s6')
mkm.nx_graph_analyze_lazy_srw(G)

In [ ]:
################ Mixing on the 2-core of the giant component of an Erdős–Rényi random graph ################

G = nx.read_sparse6('ergc_2core_2.s6')
mkm.nx_graph_analyze_lazy_srw(G)

In [ ]:
################ Mixing on a GW-Tree ################

# np.random.seed(0)

# G = grow_gw_tree(lambda: np.random.poisson(1.05, 1), num_generations = 100)
# while nx.number_of_nodes(G) < 100:
# 	G = grow_gw_tree(lambda: np.random.poisson(1.05, 1), num_generations = 100)

# #show_tree(G)

# mc = mkm.nx_graph_lazy_srw(G)
# mc.add_distributions(mkm.delta_distribution(mc.get_n(),0))
# mc.compute_tv_mixing()

# # plot the mixing
# mc.plot_tv_mixing(0)

In [ ]:
################ lazy biased random walk on the line ################

mc = mkm.MarkovChain(mkm.line_lazy_transition_matrix(100, p=0.51))
mc.add_random_delta_distributions(1)
mc.compute_tv_mixing()

mc.convergence_video('/home/sbordt/Desktop/line_biased.avi', 0, 60)

## Draw some graphs

### 10-path with attached Galton-Watson trees

In [ ]:
np.random.seed(0)

G = nx.path_graph(10)
grow_gw_trees_at_all_nodes(G, lambda: np.random.poisson(0.9, 1))

pos = nx.graphviz_layout(G, prog='neato')
nx.draw(G, pos, with_labels=False, arrows=False, node_size=100, node_color='k', edge_color='k', width=1.5)
plt.show()

pos = nx.graphviz_layout(G, prog='neato')
nx.draw(G, pos, with_labels=True, arrows=False)
plt.show()


### 2-core and path contraction for a giant component

In [ ]:
random.seed(0)

n = 100000
lam = 1.1

G = erdos_renyi_giant_component(n,lam/n)
K = pseudo_kernel(G)

print nx.number_of_nodes(G)
print nx.number_of_nodes(K)

pos = nx.graphviz_layout(K, prog='fdp')
nx.draw(K, pos, with_labels=False, arrows=False, node_size=100, node_color='k', edge_color='k', width=1.5)
plt.show()

## Coupling of random walk on two Galton-Watson trees

In [ ]:
# make the result look like a coupling of trees was tried, too
# this code does only generate the trees
np.random.seed(11)

G = grow_gw_tree(lambda: np.random.choice([1,1,1,1,1,2,2,2,3,3]), n_gen=5)
#show_tree(G)

# two nodes of degree 2 become degree 3 to reflect the different distribution
G.add_edge(21, G.number_of_nodes())
grow_gw_tree_at_node(G, G.number_of_nodes()-1, lambda: np.random.choice([1,1,1,1,1,2,2,3,3,3]), n_gen=3)

G.add_edge(35, G.number_of_nodes())
grow_gw_tree_at_node(G, G.number_of_nodes()-1, lambda: np.random.choice([1,1,1,1,1,2,2,3,3,3]), n_gen=2)

show_tree(G)

## Plot of the degree distribution of a random graph and it's giant component

In [ ]:
import matplotlib.pyplot as plt

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.stats import binom

n = 1000000
p = 1.05/n

random.seed(3)

# sampe random graph and extract it's giant component
G = nx.fast_gnp_random_graph(n,p)

number_of_nodes = 0
C_1 = 0
for c in nx.connected_component_subgraphs(G):
    if nx.number_of_nodes(c) > number_of_nodes:
        number_of_nodes = nx.number_of_nodes(c)
        C_1 = c
        
# degree distributions
d1 = np.array(nx.degree_histogram(G))
d2 = np.array(nx.degree_histogram(C_1))

print sum(d1)
print d1
print sum(d2)
print d2
print nx.number_of_nodes(C_1)

In [ ]:
# random graph
x = np.arange(8)
y = (d1/(1.0*sum(d1)))[0:8]
    
fig = plt.figure()

vmax = np.max(y)
vmin = (np.min(y)*3. - vmax)/2.

colormap = ScalarMappable(norm=Normalize(vmin, vmax), cmap='Blues')

plt.bar(x, y, color=colormap.to_rgba(y), align='edge', width=0.8)

plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')

plt.xlim(0, len(x))
plt.show()

In [ ]:
# giant component
x = np.arange(8)
y = (d2/(1.0*sum(d2)))[0:8]
    
fig = plt.figure()

vmax = np.max(y)
vmin = (np.min(y)*3. - vmax)/2.

colormap = ScalarMappable(norm=Normalize(vmin, vmax), cmap='Blues')

plt.bar(x, y, color=colormap.to_rgba(y), align='edge', width=0.8)

plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')

plt.xlim(0, len(x))
plt.show()

In [ ]:
i = 1
j = 1

print i ^ ( (i ^ (~ (i & (1 << j))) ) & (1 << j) )

In [ ]:
import scipy.sparse as ssp

def hypercube_transition_matrix(n, lazy = True):
    k = pow(2,n)
    P = ssp.lil_matrix((k,k))
    p = 1./n
    
    for i in range(0,k):
        for j in range(0,n):
            P[i, i ^ ( (i ^ (~ (i & (1 << j))) ) & (1 << j) )] = p

    if lazy:
        P = mkm.lazy(P)

    return P.tocsr()

print hypercube_transition_matrix(4,lazy=False)